# Some installation

In [0]:
!pip install pmdarima
!pip install chart_studio
!pip install pyramid-arima

# Some import and configuration

In [0]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
from datetime import datetime
import chart_studio
import chart_studio.plotly as ply
import cufflinks as cf
import plotly.graph_objects as go

In [0]:
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [0]:
# go to and get api_key you need to sign-up : https://chart-studio.plot.ly/settings/api#/
chart_studio.tools.set_credentials_file(username='***********', api_key='***********')

In [0]:
def str_to_class(str):
    return getattr(sys.modules[__name__], str)

# Total deaths, confirmed cases in the world

In [0]:
url_tot = 'https://covid.ourworldindata.org/data/ecdc/total_cases.csv'
url_deat= "https://covid.ourworldindata.org/data/ecdc/total_deaths.csv"
data_tot = pd.read_csv(url_tot,index_col=0)
data_death = pd.read_csv(url_deat,index_col=0)

In [0]:
x=[i for i in data_death.columns]

## Read any country data

In [0]:
country='China' #@param ["World","Afghanistan","Albania","Algeria","Andorra","Angola","Antigua and Barbuda","Argentina","Armenia","Australia","Austria","Azerbaijan","Bahamas","Bahrain","Bangladesh","Barbados","Belarus","Belgium","Benin","Bermuda","Bhutan","Bolivia","Bosnia and Herzegovina","Brazil","Brunei","Bulgaria","Burkina Faso","Cambodia","Cameroon","Canada","Cape Verde","Cayman Islands","Central African Republic","Chad","Chile","China","Colombia","Congo","Costa Rica","Cote d'Ivoire","Croatia","Cuba","Cyprus","Czech Republic","Democratic Republic of Congo","Denmark","Djibouti","Dominican Republic","Ecuador","Egypt","El Salvador","Equatorial Guinea","Eritrea","Estonia","Ethiopia","Faeroe Islands","Fiji","Finland","France","French Polynesia","Gabon","Gambia","Georgia","Germany","Ghana","Gibraltar","Greece","Greenland","Guam","Guatemala","Guernsey","Guinea","Guyana","Haiti","Honduras","Hungary","Iceland","India","Indonesia","International","Iran","Iraq","Ireland","Isle of Man","Israel","Italy","Jamaica","Japan","Jersey","Jordan","Kazakhstan","Kenya","Kosovo","Kuwait","Kyrgyzstan","Latvia","Lebanon","Liberia","Liechtenstein","Lithuania","Luxembourg","Macedonia","Madagascar","Malaysia","Maldives","Malta","Mauritania","Mauritius","Mexico","Moldova","Monaco","Mongolia","Montenegro","Montserrat","Morocco","Myanmar","Namibia","Nepal","Netherlands","Netherlands Antilles","New Caledonia","New Zealand","Nicaragua","Niger","Nigeria","Norway","Oman","Pakistan","Palestine","Panama","Papua New Guinea","Paraguay","Peru","Philippines","Poland","Portugal","Qatar","Romania","Russia","Rwanda","Saint Lucia","Saint Vincent and the Grenadines","San Marino","Saudi Arabia","Senegal","Serbia","Seychelles","Singapore","Slovakia","Slovenia","Somalia","South Africa","South Korea","Spain","Sri Lanka","Sudan","Suriname","Swaziland","Sweden","Switzerland","Taiwan","Tanzania","Thailand","Timor","Togo","Trinidad and Tobago","Tunisia","Turkey","Uganda","Ukraine","United Arab Emirates","United Kingdom","United States","Uruguay","Uzbekistan","Vatican","Venezuela","Vietnam","Zambia","Zimbabwe"]

## Some statistic information

In [0]:
Which_data = 'Total' #@param ["Total","Deaths"]
if(Which_data=="Total"):
  working_data=data_tot
  test_caption="Total case"
else :
  working_data=data_death
  test_caption="Death case"

In [0]:
working_data[country].describe()

In [0]:
working_data[country].plot(figsize=(15, 6))
plt.show()

In [0]:
working_data[country].iplot(title="Covid-19 "+ country)

## Preparing the data

In [0]:
#divide into train and validation set
ratio = 90 #@param {type:"slider", min:0,max:100,  step:5}
train = working_data[country][:int((ratio/100)*(len(working_data[country])))]
valid = working_data[country][int((ratio/100)*(len(working_data[country]))):]



## **1.   Arimax**




In [0]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [0]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [0]:
mod = sm.tsa.statespace.SARIMAX(train,
                                order=(1, 1, 1),
                                seasonal_order=(1 ,0,1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)  

In [0]:
results = mod.fit()
print(results.summary().tables[1])

## Prediction of train datasets

In [0]:
pred = results.get_prediction(start=pd.to_datetime('2019-12-31'), dynamic=False)
pred_ci = pred.conf_int()

In [0]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=working_data[country].index, y=working_data[country].values,
                    mode='lines',
                    name=test_caption))
fig.add_trace(go.Scatter(x=pred.predicted_mean.index, y=pred.predicted_mean.values.astype(int),
                    mode='lines',
                    name='Predictions'))
fig.show()

## Prediction of validation datasets

In [0]:
pred_uc = results.get_forecast(len(valid))
pred_ci = pred_uc.conf_int()

In [0]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=working_data[country].index, y=working_data[country].values,
                    mode='lines',
                    name=test_caption))
fig.add_trace(go.Scatter(x=pred_uc.predicted_mean.index, y=pred_uc.predicted_mean.values.astype(int),
                    mode='lines',
                    name='Predictions'))
fig.show()

##Predition of the next 15 days

In [0]:
pred_uc = results.get_forecast(len(valid)+(15-len(valid)))
pred_ci = pred_uc.conf_int()

In [0]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=working_data[country].index, y=working_data[country].values,
                    mode='lines',
                    name=test_caption))
fig.add_trace(go.Scatter(x=pred_uc.predicted_mean.index, y=pred_uc.predicted_mean.values.astype(int),
                    mode='lines',
                    name='Predictions'))
fig.show()

## **2.   Autoarima**




## Start training

In [0]:
from pmdarima import auto_arima
model = auto_arima(train, start_p=1, start_q=1,
                           max_p=2, max_q=2, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1,trace=True, error_action='ignore', suppress_warnings=True,stepwise=True)
model.fit(train)

## Prediction of the validation datatsets

In [0]:
forecast = model.predict(len(valid))

In [0]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=working_data[country].index, y=working_data[country].values,
                    mode='lines',
                    name=test_caption))
fig.add_trace(go.Scatter(x=valid.index, y=forecast.astype(int),
                    mode='lines',
                    name='Predictions'))
fig.show()

## Prediction of the next 15 days

In [0]:
date_input = '2020-03-23' #@param {type:"date"}
test = pd.date_range(datetime.strptime(date_input, '%Y-%m-%d').date(), periods=25).tolist()

In [0]:
for i in range(0,len(test)):
  test[i] = datetime.strptime(str(test[i]), '%Y-%m-%d %H:%M:%S').date()

In [0]:
forecast = model.predict(len(test))

In [0]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=working_data[country].index, y=working_data[country].values,
                    mode='lines',
                    name=test_caption))
fig.add_trace(go.Scatter(x=test, y=forecast.astype(int),
                    mode='lines',
                    name='Predictions'))
fig.show()

# Read Algeria Data

In [0]:
df = pd.read_csv("https://github.com/belarbi2733/docker_file__opencv_algo/releases/download/1/Classeur1.csv",index_col=0)
df.head()

In [0]:
df.describe()

In [0]:
index = df.index
columns = df.columns
values = df.values

In [0]:
for i in range(len(df.index)):
  df.index.values[i] = datetime.strptime(df.index[i], '%d-%m-%Y').date()


In [0]:
df.plot(figsize=(15, 6))
plt.show()

In [0]:
df.iplot(title="Covid-19 Algeria")

In [0]:
df1=df

In [0]:
#divide into train and validation set
ratio = 95 #@param {type:"slider", min:0,max:100,  step:5}
train = df1[:int((ratio/100)*(len(df1)))]
valid = df1[int((ratio/100)*(len(df1))):]

In [0]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [0]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [0]:
cases='confirmed' #@param ["deaths","recovered","confirmed"]

In [0]:
mod = sm.tsa.statespace.SARIMAX(train[cases],
                                order=(1, 1, 1),
                                seasonal_order=(1 ,0,0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)                              


In [0]:
results = mod.fit()
print(results.summary().tables[1])

In [0]:
date_input = '2020-02-25' #@param {type:"date"}
pred = results.get_prediction(start=pd.to_datetime(date_input), dynamic=False)
pred_ci = pred.conf_int()

In [0]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train[cases].index, y=train[cases].values,
                    mode='lines',
                    name=cases))
fig.add_trace(go.Scatter(x=pred.predicted_mean.index, y=pred.predicted_mean.values.astype(int),
                    mode='lines',
                    name='Predictions'))
fig.show()


In [0]:
pred_uc = results.get_forecast(steps=len(valid)+(15-len(valid)))
pred_ci = pred_uc.conf_int()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df1[cases].index, y=df1[cases].values,
                    mode='lines',
                    name=cases))
fig.add_trace(go.Scatter(x=pred_uc.predicted_mean.index, y=pred_uc.predicted_mean.values.astype(int),
                    mode='lines',
                    name='Predictions'))
fig.show()


In [0]:
predictions=pred_uc.predicted_mean
predictions.astype(int)